In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from progressBar import printProgressBar

import medicalDataLoader
import argparse
from utils import *

from UNet_Base import *
import random
import torch
import pdb

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from torchmetrics import ConfusionMatrix

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def runTraining(modelName,checkpoints=None, n_epochs=0):
    print('-' * 40)
    print('~~~~~~~~  Starting the training... ~~~~~~')
    print('-' * 40)

    ## DEFINE HYPERPARAMETERS (batch_size > 1)
    batch_size = 16
    batch_size_val = 8
    lr = 0.001
    epoch = 10
    start_epoch = 0
    if checkpoints != None :
        batch_size = checkpoints['batch_size']
        batch_size_val = checkpoints['batch_size_val']
        lr = checkpoints['lr']    # Learning Rate
        epoch = n_epochs # Number of epochs
        start_epoch = checkpoints['epoch']
        

    root_dir = './Data/'

    print(' Dataset: {} '.format(root_dir))

    ## DEFINE THE TRANSFORMATIONS TO DO AND THE VARIABLES FOR TRAINING AND VALIDATION
    
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    mask_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    train_set_full = medicalDataLoader.MedicalImageDataset('train',
                                                      root_dir,
                                                      transform=transform,
                                                      mask_transform=mask_transform,
                                                      augment=False,
                                                      equalize=False)

    train_loader_full = DataLoader(train_set_full,
                              batch_size=batch_size,
                              worker_init_fn=np.random.seed(0),
                              num_workers=0,
                              shuffle=True)
    


    val_set = medicalDataLoader.MedicalImageDataset('val',
                                                    root_dir,
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    equalize=False)

    val_loader = DataLoader(val_set,
                            batch_size=batch_size_val,
                            worker_init_fn=np.random.seed(0),
                            num_workers=0,
                            shuffle=False)


    ## INITIALIZE YOUR MODEL
    num_classes = 4 # NUMBER OF CLASSES

    print("~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~")
    modelName = 'Test_Model'
    print(" Model Name: {}".format(modelName))

    ## CREATION OF YOUR MODEL
    net = UNet(num_classes)
    if checkpoints != None and checkpoints['model_state_dict'] != None:
        net.load_state_dict = checkpoints['model_state_dict']
    

    print("Total params: {0:,}".format(sum(p.numel() for p in net.parameters() if p.requires_grad)))

    # DEFINE YOUR OUTPUT COMPONENTS (e.g., SOFTMAX, LOSS FUNCTION, ETC)
    softMax = torch.nn.Softmax()
    CE_loss = torch.nn.CrossEntropyLoss()

    ## PUT EVERYTHING IN GPU RESOURCES    
    if torch.cuda.is_available():
        net.cuda()
        softMax.cuda()
        CE_loss.cuda()

    ## DEFINE YOUR OPTIMIZER
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    if checkpoints != None and checkpoints['optimizer_state_dict'] != None:
        optimizer.load_state_dict = checkpoints['optimizer_state_dict']

    ### To save statistics ####
    lossTotalTraining = []
    Best_loss_val = 1000
    if checkpoints != None :
        Best_loss_val = checkpoints['val_loss']
    BestEpoch = 0
    
    directory = 'Results/Statistics/' + modelName

    print("~~~~~~~~~~~ Starting the training ~~~~~~~~~~")
    if os.path.exists(directory)==False:
        os.makedirs(directory)

    ## START THE TRAINING
    
    ## FOR EACH EPOCH
    for i in range(start_epoch, epoch):
        net.train()
        lossEpoch = []
        lossValEpoch = []
        DSCEpoch = []
        DSCEpoch_w = []
        num_batches = len(train_loader_full)
        num_batches_val = len(val_loader)
        
        ## FOR EACH BATCH
        for j, data in enumerate(train_loader_full):
            ### Set to zero all the gradients
            net.zero_grad()
            optimizer.zero_grad()

            ## GET IMAGES, LABELS and IMG NAMES
            images, labels, _ = data

            ### From numpy to torch variables
            labels = to_var(labels)
            # labels = torch.argmax(labels, dim=1)
            images = to_var(images)

            ################### Train ###################
            #-- The CNN makes its predictions (forward pass)
            net_predictions = net.forward(images)

            #-- Compute the losses --#
            # THIS FUNCTION IS TO CONVERT LABELS TO A FORMAT TO BE USED IN THIS CODE
            segmentation_classes = getTargetSegmentation(labels)
            
            # COMPUTE THE LOSS
            CE_loss_value = CE_loss(net_predictions, segmentation_classes) # XXXXXX and YYYYYYY are your inputs for the CE
            lossTotal = CE_loss_value
            pred = softMax(net_predictions)
            masks = torch.argmax(pred, dim=1)

            # DO THE STEPS FOR BACKPROP (two things to be done in pytorch)
            lossTotal.backward()
            optimizer.step()
            
            # THIS IS JUST TO VISUALIZE THE TRAINING 
            lossEpoch.append(lossTotal.cpu().data.numpy())
            printProgressBar(j + 1, num_batches,
                             prefix="[Training] Epoch: {} ".format(i),
                             length=15,
                             suffix=" Loss: {:.4f}, ".format(lossTotal))

            confmat = ConfusionMatrix(task="multiclass", num_classes=4)
            confmat = confmat(net_predictions, segmentation_classes).numpy()
            accuracy = np.diag(confmat).sum()/confmat.sum()

        lossEpoch = np.asarray(lossEpoch)
        lossEpoch = lossEpoch.mean()

        lossTotalTraining.append(lossEpoch)
        printProgressBar(num_batches, num_batches,
                             done="[Training] Epoch: {}, LossG: {:.4f}, Acc: {}".format(i,lossEpoch,accuracy))
        net.eval()
        for j, data_val in enumerate(val_loader):

            images_val, labels_val, _ = data_val
            labels_val = to_var(labels_val)
            images_val = to_var(labels_val)
            
            net_predictions_val = net.forward(images_val.float())

            segmentation_classes_val = getTargetSegmentation(labels_val)

            CE_loss_value_val = CE_loss(net_predictions_val, segmentation_classes_val) # XXXXXX and YYYYYYY are your inputs for the CE

            confmat_val = ConfusionMatrix(task="multiclass", num_classes=4)
            confmat_val = confmat_val(net_predictions_val, segmentation_classes_val).numpy()
            accuracy_val = np.diag(confmat_val).sum()/confmat_val.sum()

            lossValEpoch.append(CE_loss_value_val.cpu().data.numpy())
            printProgressBar(j + 1, num_batches_val,
                             prefix="[Validation] Epoch: {} ".format(i),
                             length=15,
                             suffix=" Loss: {:.4f}, ".format(CE_loss_value_val))

        lossValEpoch = np.asarray(lossValEpoch)
        lossValEpoch = lossValEpoch.mean()

        if lossValEpoch < Best_loss_val:
            Best_loss_val = lossValEpoch
            BestEpoch = i
            if not os.path.exists('./models/' + modelName):
                os.makedirs('./models/' + modelName)
            torch.save({'epoch': epoch,
                        'batch_size':batch_size,
                        'batch_size_val':batch_size_val,
                        'lr':lr,
                        'model_state_dict': net.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'train_loss': lossEpoch,
                        'val_loss': lossValEpoch,
                        }, './models/' + modelName + '/best_model')
            np.save(os.path.join(directory, 'Losses.npy'), lossTotalTraining)


        printProgressBar(num_batches_val, num_batches_val,
                             done="[Validation] Epoch: {}, LossG: {:.4f}, Acc: {}".format(i,lossValEpoch,accuracy_val))
        
def LoadTraining(modelName, n_epochs):
    if os.path.exists('./models/'+modelName+'/best_model'):
        checkpoint = torch.load('./models/'+modelName+'/best_model')
    else :
        batch_size = 16
        batch_size_val = 4
        lr = 0.001
        checkpoint = {
            'epoch': 0,
            'batch_size': batch_size,
            'batch_size_val':batch_size_val,
            'lr':lr,
            'model_state_dict': None,
            'optimizer_state_dict': None,
            'train_loss': 0,
            'val_loss': 1000,
            }
    
    runTraining(modelName, checkpoint, n_epochs)

In [6]:
runTraining('Test_model')

----------------------------------------
~~~~~~~~  Starting the training... ~~~~~~
----------------------------------------
 Dataset: ./Data/ 
~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~
 Model Name: Test_Model
Total params: 60,664
~~~~~~~~~~~ Starting the training ~~~~~~~~~~
[Training] Epoch: 0 [DONE]                                 
[Training] Epoch: 0, LossG: 1.9054, Acc: 0.044680277506510414                                                
[Validation] Epoch: 0 [DONE]                                 
[Validation] Epoch: 0, LossG: 1.4326, Acc: 0.027099609375                                                    
[Training] Epoch: 1 [DONE]                                 
[Training] Epoch: 1, LossG: 1.7737, Acc: 0.03450139363606771                                                 
[Validation] Epoch: 1 [DONE]                                 
[Validation] Epoch: 1, LossG: 1.4144, Acc: 0.16558074951171875                                               
[Training] Epoch: 2 [DONE]          